In [1]:
cd ../..

C:\Users\thinh\Documents\TN-Vathes-GoogleCloud\CODE\DataJoint\datajoint-python


C:\Users\thinh\.conda\envs\datajoint-python\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import datajoint as dj
from pathlib import Path

In [3]:
dj.config["database.host"] = "localhost"
dj.config["database.user"] = "root"
dj.config["database.password"] = "simple"
dj.conn()

[2024-07-16 19:22:44,426][INFO]: Connecting root@localhost:3306
[2024-07-16 19:22:44,579][INFO]: Connected root@localhost:3306


DataJoint connection (connected) root@localhost:3306

In [4]:
# set experimental feature to true: DJ_SUPPORT_FILESET_MANAGEMENT
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = "TRUE"
os.environ['DJ_SUPPORT_FILESET_MANAGEMENT'] = "TRUE"

In [5]:
# configure an external store
STORE_NAME = 'file-store'
dj.config['stores'] = {STORE_NAME: {
    'protocol': 'file',
    'location': 'C:/Users/thinh/Documents/SciOps-data/datajoint-python/inbox',
    'stage': 'C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox'}
}

In [6]:
schema = dj.Schema('test_fileset')

In [7]:
@schema
class Session(dj.Manual):
    definition = """
    session_id: int
    ---
    session_fileset: fileset@file-store
    """

In [8]:
data_dir = Path(r"C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures")

In [9]:
# Let check the contents of the `data_dir`
list(data_dir.glob("*"))

[WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r396_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r538_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/o

#### Now, insert as a `fileset` datatype

`fileset` type can be inserted from a directory

In [10]:
Session.insert1((0, data_dir))

In [11]:
Session()

session_id,session_fileset
0,=BLOB=


In [12]:
(Session & {'session_id': 0}).fetch1('session_fileset')

['C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r396_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r397_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r498_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r503_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r506_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r535_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r538_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r539_p

#### We can also check the contents of the schema-level `fileset` store

In [13]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
ef052cdd-7714-e39a-0d1c-662eb6ebe9ea,report_figures,704886,11,2024-07-17 00:22:58


In [14]:
schema.fileset['file-store'].File

datajoint.external.FileSetTable.FileTable

In [15]:
schema.fileset['file-store'].File

datajoint.external.FileSetTable.FileTable

In [16]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
0549ed90-1bd8-8ed8-8ce6-17062ab756e0,66674,None,report_figures/m37_m37c2000r650_psychometric_curve.png,f79642d0-baa9-b410-811f-990563290413,2024-07-17 00:22:58
4797c923-e2ce-c7d9-797e-fdb34e93a8f2,69809,None,report_figures/m37_m37c2000r397_psychometric_curve.png,60684f90-512e-2266-f1b1-3a9988fb625c,2024-07-17 00:22:58
4c9bcfc2-914b-f301-d3e6-a428c86411d9,85124,None,report_figures/m37_m37c2000r396_psychometric_curve.png,7cceba60-a79e-213d-bb26-a2364bfedf55,2024-07-17 00:22:58
6169fa12-4bea-6de4-0d4e-9a04bbb20929,68675,None,report_figures/m37_m37c212r7_psychometric_curve.png,aad2ee9f-8fbd-87df-cc7d-2a74302fbbf4,2024-07-17 00:22:58
789aa3ef-18a7-75a7-f735-89dfba8c1688,68996,None,report_figures/m37_m37c2000r539_psychometric_curve.png,25ae864c-1c2c-5f0c-1a9a-7cfaebe9097c,2024-07-17 00:22:58
7a31742e-db74-4325-732f-db762d351fdf,68334,None,report_figures/m37_m37c2000r649_psychometric_curve.png,10a7a0a7-3e3a-6656-ad2b-bc177107e4be,2024-07-17 00:22:58
7b7a16db-a81b-d606-1adb-322adadc403e,43696,None,report_figures/m37_m37c2000r538_psychometric_curve.png,2963bed7-1033-350d-44a4-98ecb9c55e73,2024-07-17 00:22:58
9e86bdfa-91db-3278-288c-05c6442135f6,42620,None,report_figures/m37_m37c2000r535_psychometric_curve.png,ddd08a40-a152-8262-4a96-c75b4d448300,2024-07-17 00:22:58
9eec6c50-2061-5261-d04b-27223ce85da5,67442,None,report_figures/m37_m37c2000r506_psychometric_curve.png,e0c817b1-45cd-47f6-b431-a568e648730c,2024-07-17 00:22:58
a8409d57-729c-ef9b-1558-882890c448d9,57842,None,report_figures/m37_m37c2000r503_psychometric_curve.png,6daebf37-867a-db5b-288e-f70b28e1a60e,2024-07-17 00:22:58


#### Now, let's add another entry into `Session` with the same fileset

In [17]:
Session.insert1((1, data_dir))

In [18]:
Session()

session_id,session_fileset
0,=BLOB=
1,=BLOB=


#### Check the external fileset again, we should not expect file duplications

In [19]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
0549ed90-1bd8-8ed8-8ce6-17062ab756e0,66674,None,report_figures/m37_m37c2000r650_psychometric_curve.png,f79642d0-baa9-b410-811f-990563290413,2024-07-17 00:22:58
4797c923-e2ce-c7d9-797e-fdb34e93a8f2,69809,None,report_figures/m37_m37c2000r397_psychometric_curve.png,60684f90-512e-2266-f1b1-3a9988fb625c,2024-07-17 00:22:58
4c9bcfc2-914b-f301-d3e6-a428c86411d9,85124,None,report_figures/m37_m37c2000r396_psychometric_curve.png,7cceba60-a79e-213d-bb26-a2364bfedf55,2024-07-17 00:22:58
6169fa12-4bea-6de4-0d4e-9a04bbb20929,68675,None,report_figures/m37_m37c212r7_psychometric_curve.png,aad2ee9f-8fbd-87df-cc7d-2a74302fbbf4,2024-07-17 00:22:58
789aa3ef-18a7-75a7-f735-89dfba8c1688,68996,None,report_figures/m37_m37c2000r539_psychometric_curve.png,25ae864c-1c2c-5f0c-1a9a-7cfaebe9097c,2024-07-17 00:22:58
7a31742e-db74-4325-732f-db762d351fdf,68334,None,report_figures/m37_m37c2000r649_psychometric_curve.png,10a7a0a7-3e3a-6656-ad2b-bc177107e4be,2024-07-17 00:22:58
7b7a16db-a81b-d606-1adb-322adadc403e,43696,None,report_figures/m37_m37c2000r538_psychometric_curve.png,2963bed7-1033-350d-44a4-98ecb9c55e73,2024-07-17 00:22:58
9e86bdfa-91db-3278-288c-05c6442135f6,42620,None,report_figures/m37_m37c2000r535_psychometric_curve.png,ddd08a40-a152-8262-4a96-c75b4d448300,2024-07-17 00:22:58
9eec6c50-2061-5261-d04b-27223ce85da5,67442,None,report_figures/m37_m37c2000r506_psychometric_curve.png,e0c817b1-45cd-47f6-b431-a568e648730c,2024-07-17 00:22:58
a8409d57-729c-ef9b-1558-882890c448d9,57842,None,report_figures/m37_m37c2000r503_psychometric_curve.png,6daebf37-867a-db5b-288e-f70b28e1a60e,2024-07-17 00:22:58


In [20]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
ef052cdd-7714-e39a-0d1c-662eb6ebe9ea,report_figures,704886,11,2024-07-17 00:22:58


In [21]:
schema.fileset['file-store'].File

datajoint.external.FileSetTable.FileTable

#### Next, let's introduce a new fileset - we'll use a subset of the files within the data directory

In [22]:
fileset2 = list(Path(data_dir).glob("*c2000*"))

In [23]:
fileset2

[WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r396_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r538_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/o

#### One more time - we'll use the same files from "data_dir" plus one more file

In [24]:
data_dir

WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures')

In [25]:
list(Path(data_dir.parent).glob("*"))

[WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/bistab'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures')]

In [26]:
fileset3 = list(data_dir.rglob("*")) + [Path('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/bistab/bistab_0001.h5')]

In [27]:
fileset3

[WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r396_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/outbox/report_figures/m37_m37c2000r538_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/SciOps-data/datajoint-python/o

#### Insert new record into Session - `fileset` type can be inserted as a list of paths

In [28]:
Session.insert1((2, fileset2))

In [29]:
Session.insert1((3, fileset3))

In [31]:
Session()

session_id,session_fileset
2,=BLOB=
3,=BLOB=
0,=BLOB=
1,=BLOB=


#### Let's check the external fileset store again

In [32]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
6291268b-0b70-1077-465d-40fb0ab1ccb3,report_figures,636211,10,2024-07-17 00:23:16
b6507659-f067-f608-e7e4-4708e9c78f5c,.,8001582,12,2024-07-17 00:23:17
ef052cdd-7714-e39a-0d1c-662eb6ebe9ea,report_figures,704886,11,2024-07-17 00:22:58


In [33]:
schema.fileset['file-store'].File()

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,file_hash hash of relative filepath (filepath),"filepath filepath relative to the ""location"" of the store",file
6291268b-0b70-1077-465d-40fb0ab1ccb3,0549ed90-1bd8-8ed8-8ce6-17062ab756e0,report_figures/m37_m37c2000r650_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,4797c923-e2ce-c7d9-797e-fdb34e93a8f2,report_figures/m37_m37c2000r397_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,4c9bcfc2-914b-f301-d3e6-a428c86411d9,report_figures/m37_m37c2000r396_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,789aa3ef-18a7-75a7-f735-89dfba8c1688,report_figures/m37_m37c2000r539_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,7a31742e-db74-4325-732f-db762d351fdf,report_figures/m37_m37c2000r649_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,7b7a16db-a81b-d606-1adb-322adadc403e,report_figures/m37_m37c2000r538_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,9e86bdfa-91db-3278-288c-05c6442135f6,report_figures/m37_m37c2000r535_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,9eec6c50-2061-5261-d04b-27223ce85da5,report_figures/m37_m37c2000r506_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,a8409d57-729c-ef9b-1558-882890c448d9,report_figures/m37_m37c2000r503_psychometric_curve.png,=BLOB=
6291268b-0b70-1077-465d-40fb0ab1ccb3,e8fba290-bd1a-b422-fed1-12ae0dd0357e,report_figures/m37_m37c2000r498_psychometric_curve.png,=BLOB=


In [34]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
0549ed90-1bd8-8ed8-8ce6-17062ab756e0,66674,None,report_figures/m37_m37c2000r650_psychometric_curve.png,f79642d0-baa9-b410-811f-990563290413,2024-07-17 00:22:58
4797c923-e2ce-c7d9-797e-fdb34e93a8f2,69809,None,report_figures/m37_m37c2000r397_psychometric_curve.png,60684f90-512e-2266-f1b1-3a9988fb625c,2024-07-17 00:22:58
4c9bcfc2-914b-f301-d3e6-a428c86411d9,85124,None,report_figures/m37_m37c2000r396_psychometric_curve.png,7cceba60-a79e-213d-bb26-a2364bfedf55,2024-07-17 00:22:58
6169fa12-4bea-6de4-0d4e-9a04bbb20929,68675,None,report_figures/m37_m37c212r7_psychometric_curve.png,aad2ee9f-8fbd-87df-cc7d-2a74302fbbf4,2024-07-17 00:22:58
789aa3ef-18a7-75a7-f735-89dfba8c1688,68996,None,report_figures/m37_m37c2000r539_psychometric_curve.png,25ae864c-1c2c-5f0c-1a9a-7cfaebe9097c,2024-07-17 00:22:58
7a31742e-db74-4325-732f-db762d351fdf,68334,None,report_figures/m37_m37c2000r649_psychometric_curve.png,10a7a0a7-3e3a-6656-ad2b-bc177107e4be,2024-07-17 00:22:58
7b7a16db-a81b-d606-1adb-322adadc403e,43696,None,report_figures/m37_m37c2000r538_psychometric_curve.png,2963bed7-1033-350d-44a4-98ecb9c55e73,2024-07-17 00:22:58
9e86bdfa-91db-3278-288c-05c6442135f6,42620,None,report_figures/m37_m37c2000r535_psychometric_curve.png,ddd08a40-a152-8262-4a96-c75b4d448300,2024-07-17 00:22:58
9eec6c50-2061-5261-d04b-27223ce85da5,67442,None,report_figures/m37_m37c2000r506_psychometric_curve.png,e0c817b1-45cd-47f6-b431-a568e648730c,2024-07-17 00:22:58
a8409d57-729c-ef9b-1558-882890c448d9,57842,None,report_figures/m37_m37c2000r503_psychometric_curve.png,6daebf37-867a-db5b-288e-f70b28e1a60e,2024-07-17 00:22:58


#### No duplication of files on external store (thanks to the feature of "attach" type)

In [37]:
(Session & "session_id = 2").fetch1('session_fileset')

['C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r396_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r397_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r498_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r503_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r506_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r535_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r538_psychometric_curve.png',
 'C:\\Users\\thinh\\Documents\\SciOps-data\\datajoint-python\\outbox\\report_figures\\m37_m37c2000r539_p

#### Drop everything and clean up

In [38]:
Session.delete()

[2024-07-16 19:24:22,841][INFO]: Deleting 4 rows from `test_fileset`.`session`


Commit deletes? [yes, No]: yes


[2024-07-16 19:24:24,511][INFO]: Deletes committed.


4

In [42]:
schema.fileset['file-store'].unused()

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
6291268b-0b70-1077-465d-40fb0ab1ccb3,report_figures,636211,10,2024-07-17 00:23:16
b6507659-f067-f608-e7e4-4708e9c78f5c,.,8001582,12,2024-07-17 00:23:17
ef052cdd-7714-e39a-0d1c-662eb6ebe9ea,report_figures,704886,11,2024-07-17 00:22:58


In [43]:
schema.fileset['file-store'].delete()

In [44]:
schema.external['file-store'].unused()

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
0549ed90-1bd8-8ed8-8ce6-17062ab756e0,66674,None,report_figures/m37_m37c2000r650_psychometric_curve.png,f79642d0-baa9-b410-811f-990563290413,2024-07-17 00:22:58
4797c923-e2ce-c7d9-797e-fdb34e93a8f2,69809,None,report_figures/m37_m37c2000r397_psychometric_curve.png,60684f90-512e-2266-f1b1-3a9988fb625c,2024-07-17 00:22:58
4c9bcfc2-914b-f301-d3e6-a428c86411d9,85124,None,report_figures/m37_m37c2000r396_psychometric_curve.png,7cceba60-a79e-213d-bb26-a2364bfedf55,2024-07-17 00:22:58
6169fa12-4bea-6de4-0d4e-9a04bbb20929,68675,None,report_figures/m37_m37c212r7_psychometric_curve.png,aad2ee9f-8fbd-87df-cc7d-2a74302fbbf4,2024-07-17 00:22:58
789aa3ef-18a7-75a7-f735-89dfba8c1688,68996,None,report_figures/m37_m37c2000r539_psychometric_curve.png,25ae864c-1c2c-5f0c-1a9a-7cfaebe9097c,2024-07-17 00:22:58
7a31742e-db74-4325-732f-db762d351fdf,68334,None,report_figures/m37_m37c2000r649_psychometric_curve.png,10a7a0a7-3e3a-6656-ad2b-bc177107e4be,2024-07-17 00:22:58
7b7a16db-a81b-d606-1adb-322adadc403e,43696,None,report_figures/m37_m37c2000r538_psychometric_curve.png,2963bed7-1033-350d-44a4-98ecb9c55e73,2024-07-17 00:22:58
9e86bdfa-91db-3278-288c-05c6442135f6,42620,None,report_figures/m37_m37c2000r535_psychometric_curve.png,ddd08a40-a152-8262-4a96-c75b4d448300,2024-07-17 00:22:58
9eec6c50-2061-5261-d04b-27223ce85da5,67442,None,report_figures/m37_m37c2000r506_psychometric_curve.png,e0c817b1-45cd-47f6-b431-a568e648730c,2024-07-17 00:22:58
a8409d57-729c-ef9b-1558-882890c448d9,57842,None,report_figures/m37_m37c2000r503_psychometric_curve.png,6daebf37-867a-db5b-288e-f70b28e1a60e,2024-07-17 00:22:58


In [45]:
schema.external['file-store'].delete(delete_external_files=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 45.99it/s]


[]